# Geobarometer & Liquidus Phase Diagram Tests
Guil supplied Excel file for testing. Output:
- Generate phases (rows) P (columns), entries are liquidus temperatures, plus geobarometer (T,P)
- P intervals should be 25 MPa; log10 fO2 intervals
- Generate an initial condition tab similar to the rhyolite-MELTS Workbook

In [ ]:
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
import sys
import pandas as pd
import time
from openpyxl import Workbook
import matplotlib.pyplot as plt
%matplotlib inline

Required ThermoEngine packages

In [ ]:
from thermoengine import core, phases, model, equilibrate
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

Use rhyolite-MELTS 1.0.2 liquid as the omnicomponent phase

In [ ]:
src_obj = core.get_src_object('EquilibrateUsingMELTSv102')
modelDB = model.Database(liq_mod='v1.0')
Liquid = modelDB.get_phase('Liq')

Specify a collection of compatible solid phases

In [ ]:
Feldspar = modelDB.get_phase('Fsp')
Quartz = modelDB.get_phase('Qz')
Spinel = modelDB.get_phase('SplS')
Opx = modelDB.get_phase('Opx')
RhomOx = modelDB.get_phase('Rhom')
Water = phases.PurePhase('WaterMelts', 'H2O', calib=False)

Set up conversion matrices for translating solid phase (endmember) stoichiometry to liquid components

In [ ]:
elmLiqMat = Liquid.props['element_comp']
idx = np.argwhere(np.all(elmLiqMat[..., :] == 0, axis=0))
conLiqMat = np.linalg.inv(np.delete(elmLiqMat, idx, axis=1))
conLiqMat[np.abs(conLiqMat) < np.finfo(np.float).eps] = 0
conPhs_d = {}
for phs in [Quartz, Feldspar, Spinel, Opx, RhomOx]:
    name = phs.props['phase_name']
    elmPhsMat = phs.props['element_comp']
    elmPhsMat = (np.delete(elmPhsMat, idx, axis=1))
    conPhsMat = np.matmul(elmPhsMat, conLiqMat)
    conPhsMat[np.abs(conPhsMat) < 100*np.finfo(np.float).eps] = 0
    conPhs_d[name] = conPhsMat

## Set up functions

In [ ]:
elm_sys = ['H','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni']
phs_sys = [Liquid, Water]
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

In [ ]:
def wt_oxides_to_elements(oxide_d):
    tot_grm_oxides = 0.0
    for key in oxide_d.keys():
        tot_grm_oxides += oxide_d[key]
    mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
    moles_end,oxide_res = Liquid.calc_endmember_comp(
        mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
    if not Liquid.test_endmember_comp(moles_end):
        print ("Calculated composition is infeasible!")
    mol_elm = Liquid.covert_endmember_comp(moles_end,output='moles_elements')
    blk_cmp = []
    for elm in elm_sys:
        index = core.chem.PERIODIC_ORDER.tolist().index(elm)
        blk_cmp.append(mol_elm[index])
    return np.array(blk_cmp)
def affinity(x, blk_cmp, NNO_offset, doprint=False, return_for_scipy=True):
    global state, equil, Feldspar, Quartz
    t = x[0] 
    p = x[1]
    if state is None:
        state = equil.execute(t, p, bulk_comp=blk_cmp, con_deltaNNO=NNO_offset, debug=0, stats=False)
    else:
        state = equil.execute(t, p, state=state, con_deltaNNO=NNO_offset, debug=0, stats=False)
    if doprint:
        state.print_state()
    muLiq = state.dGdn(t=t, p=p, element_basis=False)[:,0]
    result = {}
    muFld = np.array([
        5.0*muLiq[0]/2.0 + muLiq[2]/2.0 + muLiq[11]/2.0,
            muLiq[0] + muLiq[2] + muLiq[10],
        2.0*muLiq[0] + muLiq[12]
    ])
    muQtz = np.array([muLiq[0]])
    result['Feldspar'] = Feldspar.affinity_and_comp(t, p, muFld, method='special')
    result['Quartz']   = Quartz.affinity_and_comp(t, p, muQtz, method='special')
    if return_for_scipy:
        AffnFld = result['Feldspar'][0]
        AffnQtz = result['Quartz'][0]
        sumsq = (AffnFld/13.0)**2 + (AffnQtz/3.0)**2
        print ('x', end='')
        return sumsq
    else:
        return result
def saturation_curve(t, p, blk_cmp, NNO_offset, phase_obj, doprint=False, return_for_scipy=True):
    global state, equil, conPhs_d
    if state is None:
        state = equil.execute(t, p, bulk_comp=blk_cmp, con_deltaNNO=NNO_offset, debug=0, stats=False)
    else:
        state = equil.execute(t, p, state=state, con_deltaNNO=NNO_offset, debug=0, stats=False)
    if doprint:
        state.print_state()
    muLiq = state.dGdn(t=t, p=p, element_basis=False)[:,0]
    mLiq  = state.compositions(phase_name='Liquid')
    phase_name = phase_obj.props['phase_name']
    assert phase_name in conPhs_d, phase_name+" is not in conPhs_d"
    muPhase = []
    conPhsMat = conPhs_d[phase_name]
    for i in range(0,conPhsMat.shape[0]):
        sum = 0.0
        for j in range (0,conPhsMat.shape[1]):
            if conPhsMat[i,j] != 0 and mLiq[j] == 0:
                sum = 0.0
                break
            sum += conPhsMat[i,j]*muLiq[j]
        muPhase.append(sum)
    muPhase = np.array(muPhase)
    result = phase_obj.affinity_and_comp(t, p, muPhase, method='special')
    if return_for_scipy:
        return result[0] 
    else:
        return result

## Iterate through data file

In [ ]:
CSV_Filename = 'MELTS_Excel-2020Dec16-TP_WR_NNOvar_Python.csv'
df = pd.read_csv(CSV_Filename, index_col=0, header=None).T
df = df.fillna(0)

In [ ]:
gb_fun = []
gb_NNO = []
gb_T = []
gb_P = []
phases_l = []
time_start = time.time()
time_gb_total = 0.0
time_lq_total = 0.0
phase_names = [phs.props['phase_name'] for phs in [Quartz, Feldspar, Spinel, Opx, RhomOx]]
pressures = [(5000-ip*250)/10.0 for ip in range(0,17)]
for index, row in df.iterrows():
    grm_oxides = {
        'SiO2':  float(row['SiO2']), 
        'TiO2':  float(row['TiO2']), 
        'Al2O3': float(row['Al2O3']), 
        'Fe2O3': float(row['Fe2O3']),
        'Cr2O3': float(row['Cr2O3']), 
        'FeO':   float(row['FeO']), 
        'MnO':   float(row['MnO']),
        'MgO':   float(row['MgO']), 
        'NiO':   float(row['NiO']), 
        'CoO':   float(row['CoO']),
        'CaO':   float(row['CaO']), 
        'Na2O':  float(row['Na2O']), 
        'K2O':   float(row['K2O']), 
        'P2O5':  float(row['P2O5']), 
        'H2O':   10.0
    }
    blk_cmp = wt_oxides_to_elements(grm_oxides)
    # geothermobarometer
    t = 1034.0
    p = 1750.0
    NNO_offset = float(row['fO2 offset'])
    state = None
    time_before = time.time()
    result = opt.minimize(affinity, 
                          np.array([t, p]), 
                          args=(blk_cmp, NNO_offset),
                          options={'disp':False, 'xatol':1.0, 'fatol':1.0, 'return_all':False},
                          method='Nelder-Mead'
                         )
    time_after = time.time()
    print ('')
    print ('Ind: {0:4d} delta NNO: {1:5.2f} func: {2:13.6g} T (°C): {3:8.2f} P (MPa): {4:8.2f} time: {5:8.2f} secs'.format(
        index, NNO_offset, result.fun, result.x[0]-273.15, result.x[1]/10.0, time_after-time_before))
    gb_fun.append(result.fun)
    gb_NNO.append(NNO_offset)
    gb_T.append(result.x[0])
    gb_P.append(result.x[1])
    time_gb_total += time_after-time_before
    # liquidus phase diagram
    y = []
    time_before = time_after
    for phs in [Quartz, Feldspar, Spinel, Opx, RhomOx]: 
        x = []
        tk = t
        state = None
        print (phs.props['phase_name'])
        for ip in range(0,17):
            result = opt.root_scalar(saturation_curve, 
                                     bracket=(500,2000), 
                                     x0=tk, x1=tk-25, 
                                     xtol=.1, 
                                     args=(5000-ip*250, blk_cmp, NNO_offset, phs), 
                                     method='secant') # secant is fastest
            tk = result.root
            x.append(tk-273.15)
            print ('{0:6.1f}'.format(tk-273.15), end=' ')
        print ()
        y.append(x)
    phases_l.append(y)
    time_after = time.time()
    print ('Liquidus calculation time:', time_after-time_before, 'secs')
    time_lq_total += time_after-time_before
print ('Toal geothermobarometer execution time', time_gb_total, 'secs')
print ('Toal liquidus phase diagram execution time', time_lq_total, 'secs')
print ('Toal execution time', time_after - time_start, 'secs')

In [ ]:
plt.plot(np.array(gb_P), np.array(gb_fun), 'r+')

In [ ]:
#gb_fun
#gb_NNO
#gb_T
#gb_P
#phases_l[0]
#phase_names
#pressures

In [ ]:
def start_excel_workbook_with_sheet_name(sheetName="Summary"):
    wb = Workbook()
    ws = wb.active
    ws.title = sheetName
    row = 0
    return wb
def write_excel_workbook(wb, fileName="junk.xlsx"):
    wb.save(filename = fileName)
def write_to_cell_in_sheet_row_col(ws, row, col, value, format='general'):
    if format == 'number':
        ws.cell(row=row, column=col, value=float(value)).number_format = '0.00'
    elif format == 'scientific':
        ws.cell(row=row, column=col, value=float(value)).number_format = '0.00E+00'
    else:
        ws.cell(row=row, column=col, value=value)
def update_melts_excel_output_workbook(wb, comp_dict, params):
    condSh = wb["init_cond"]
    # NEED TO GET OXIDE COMPOSITION OF SYSTEM
    oxides = [i for i in comp_dict.keys()]
    for i in range(0,len(oxides)):
        write_to_cell_in_sheet_row_col(condSh, i+1, 1, oxides[i])
        write_to_cell_in_sheet_row_col(condSh, i+1, 2, comp_dict.get(oxides[i]))

        write_to_cell_in_sheet_row_col(condSh, 1, 4,  "T1 (C)")
        write_to_cell_in_sheet_row_col(condSh, 2, 4,  "T2 (C)")
        write_to_cell_in_sheet_row_col(condSh, 3, 4,  "T step (C)")
        write_to_cell_in_sheet_row_col(condSh, 5, 4, "P1 (MPa)")
        write_to_cell_in_sheet_row_col(condSh, 6, 4, "P2 (MPa)")
        write_to_cell_in_sheet_row_col(condSh, 7, 4, "P step (MPa)")
        write_to_cell_in_sheet_row_col(condSh, 9, 4, "fO2 value")
        write_to_cell_in_sheet_row_col(condSh, 10, 4, "fO2 buffer")

        write_to_cell_in_sheet_row_col(condSh, 1, 5, params[0])
        write_to_cell_in_sheet_row_col(condSh, 2, 5, params[1])
        write_to_cell_in_sheet_row_col(condSh, 3, 5, params[2])
        write_to_cell_in_sheet_row_col(condSh, 5, 5, params[3])
        write_to_cell_in_sheet_row_col(condSh, 6, 5, params[4])
        write_to_cell_in_sheet_row_col(condSh, 7, 5, params[5])
        write_to_cell_in_sheet_row_col(condSh, 9, 5, params[6])
        write_to_cell_in_sheet_row_col(condSh, 10, 5, params[7])

        CalcType = params[9]
        MELTS_Model = params[10]

        if CalcType == 'HP_Sequence':
            write_to_cell_in_sheet_row_col(condSh, 12, 4, "H step (kJ)")
            write_to_cell_in_sheet_row_col(condSh, 12, 5, delta_H)

        if CalcType == "TV_Sequence":
            write_to_cell_in_sheet_row_col(condSh, 12, 4, "V step (cm3)")
            write_to_cell_in_sheet_row_col(condSh, 12, 5, delta_V)

        if CalcType == "SP_Sequence":
            write_to_cell_in_sheet_row_col(condSh, 12, 4, "S step (kJ)")
            write_to_cell_in_sheet_row_col(condSh, 12, 5, delta_S)

        write_to_cell_in_sheet_row_col(condSh, 1, 7, "Model")
        write_to_cell_in_sheet_row_col(condSh, 2, 7, "Calculation")
        write_to_cell_in_sheet_row_col(condSh, 1, 8, "rhyolite-Melts_v"+MELTS_Model[:-2]+".x")
        write_to_cell_in_sheet_row_col(condSh, 2, 8, CalcType)

        if (CalcType == "Draw_Phase_Diagram" and Include_P_Calc == True):
            write_to_cell_in_sheet_row_col(condSh, 2, 8, "QF_P_Calc")

In [ ]:
comps = pd.read_csv(CSV_Filename).replace(np.nan, 0, regex=True).set_index(' ', drop = True)
model_working_dir = "Excel_results"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
for index,c in enumerate(comps):
    comp = comps[c]
    composition = comp[:15]
    composition = pd.to_numeric(composition).to_dict()
    T1 = float(comp['T1'])
    T2 = float(comp['T2'])
    delta_T = float(comp['?T'])
    P1 = float(comp['P1'])
    P2 = float(comp['P2'])
    delta_P = float(comp['?P'])
    const_fO2 = comp['fO2 constraint']
    fO2Path = comp['fO2 buffer']
    fO2_offset = float(comp['fO2 offset'])
    # parameters for modeling to be saved
    params = [T1, T2, delta_T, P1, P2, delta_P, fO2_offset, fO2Path, const_fO2]
    print (c)
    CalcType = comp['Calculation']
    params.append(CalcType)
    MELTS_Model = comp['Model']
    # the below might be unnecessary
    if MELTS_Model == '1.0.x':
        MELTS_Model = '1.0.2'
    elif MELTS_Model == '1.2.x':
        MELTS_Model = '1.2.0'
    elif MELTS_Model == '5.6.x':
        MELTS_Model = '5.6.1'
    elif MELTS_Model == '1.1.x':
        MELTS_Model = '1.1.0'
    params.append(MELTS_Model)
    # initial conditions
    wb = start_excel_workbook_with_sheet_name(sheetName="init_cond")
    update_melts_excel_output_workbook(wb, composition, params)
    # results
    ws = wb.create_sheet('Summary')
    #wb.active = ws
    write_to_cell_in_sheet_row_col(ws, 1, 1, 'Phase')
    for ip, p in enumerate(pressures):
        write_to_cell_in_sheet_row_col(ws, 1, 1+ip+1, p, format='number')
    sample = phases_l[index]
    for irow,row in enumerate(sample):
        write_to_cell_in_sheet_row_col(ws, 1+irow+1, 1, phase_names[irow], format='general')
        for ivalue,value in enumerate(row):
            write_to_cell_in_sheet_row_col(ws, 1+irow+1, 1+ivalue+1, value, format='number')
    write_to_cell_in_sheet_row_col(ws, 1+irow+3, 1, 'T (°C) Geobarometer', format='general')
    write_to_cell_in_sheet_row_col(ws, 1+irow+3, 2, gb_T[index]-273.15, format='number')
    write_to_cell_in_sheet_row_col(ws, 1+irow+4, 1, 'P (MPa) Geobarometer', format='general')
    write_to_cell_in_sheet_row_col(ws, 1+irow+4, 2, gb_P[index]/10.0, format='number')
    write_to_cell_in_sheet_row_col(ws, 1+irow+5, 1, 'Func Geobarometer', format='general')
    write_to_cell_in_sheet_row_col(ws, 1+irow+5, 2, gb_fun[index], format='scientific')
    # write workbook
    write_excel_workbook(wb, fileName=c+'.xlsx')

In [ ]:
%cd ..

In [ ]:
import multiprocessing
argument_t = ()
for index, row in df.iterrows():
    grm_oxides = {
        'SiO2':  float(row['SiO2']), 
        'TiO2':  float(row['TiO2']), 
        'Al2O3': float(row['Al2O3']), 
        'Fe2O3': float(row['Fe2O3']),
        'Cr2O3': float(row['Cr2O3']), 
        'FeO':   float(row['FeO']), 
        'MnO':   float(row['MnO']),
        'MgO':   float(row['MgO']), 
        'NiO':   float(row['NiO']), 
        'CoO':   float(row['CoO']),
        'CaO':   float(row['CaO']), 
        'Na2O':  float(row['Na2O']), 
        'K2O':   float(row['K2O']), 
        'P2O5':  float(row['P2O5']), 
        'H2O':   10.0
    }
    blk_cmp = wt_oxides_to_elements(grm_oxides)
    t = 1034.0
    p = 1750.0
    NNO_offset = float(row['fO2 offset'])
    argument_t += ([index, blk_cmp, t, p, NNO_offset],)

In [ ]:
results = None
def mp_worker(A):
    global state
    index      = A[0]
    blk_cmp    = A[1]
    t          = A[2]
    p          = A[3]
    NNO_offset = A[4]
    state = None
    result = opt.minimize(affinity, 
                          np.array([t, p]), 
                          args=(blk_cmp, NNO_offset),
                          options={'disp':False, 'xatol':1.0, 'fatol':1.0, 'return_all':False},
                          method='Nelder-Mead'
                         )
    return (index, result.fun, result.x[0], result.x[1])
def mp_handler():
    global results
    pool = multiprocessing.Pool(4)
    results = pool.map(mp_worker, argument_t)

In [ ]:
if __name__ == '__main__':
    time_before = time.time()
    mp_handler()
    time_after = time.time()
    print ("Execution time:", time_after-time_before)

In [ ]:
results